# 4. Preparación de datos
*"Es un error capital teorizar antes de tener datos. Sin darse cuenta, uno empieza a deformar los hechos para que se ajusten a las teorías, en lugar de ajustar las teorías a los hechos."  
Arthur Conan Doyle en "Escándalo en Bohemia".*

## 4.1 Selección de filas y columnas
El archivo "hoteles-vienna.xlsx" contiene información de hoteles con habitaciones disponibles para cierto fin de semana. Aunque tenemos varias variables, solamente utilizaremos las siguientes:
- hotel_id: es un identificador que reemplaza el nombre del hotel por razones de confidencialidad.
- accommodationtype: tipo de hospedaje
- price: precio.
- center1distance: distancia al centro
- starrating: calificación de 1 a 5 estrellas (más es mejor). 
- guestreviewsrating: calificación promedio otorgada por los clientes

In [1]:
# Importa la biblioteca de pandas
import pandas as pd

In [2]:
# Carga el archivo 'data/b05a_hoteles-viena.xlsx'
variables = ["hotel_id", "accommodationtype", "price", "center1distance",
             "starrating", "guestreviewsrating"]
original_df = pd.read_excel('data/b05a_hoteles-viena.xlsx', usecols=variables)

Aunque es común utilizar `df` como nombre de un DataFrame, es recomendable utilizar nombres descriptivos que nos permitan distinguir si los datos son originales, seleccionados o procesados.

In [3]:
original_df.head()

,center1distance,price,starrating,accommodationtype,guestreviewsrating,hotel_id
0,2.7,81,4.0,Apartment,4.45,21894
1,1.7,81,4.0,Hotel,3.95,21897
2,1.4,85,4.0,Hotel,3.75,21901
3,1.7,83,3.0,Hotel,45.00,21902
4,1.2,82,4.0,Hotel,3.95,21903


La base de datos abarca diferentes tipos de hospedajes. Mediante `df['A'].value_counts()` podemos consultar qué categorías y frecuencias tiene una variable 'A' en un dataframe llamado 'df'.

In [4]:
# Obtén la frecuencia por tipo de hospedaje
original_df['accommodationtype'].value_counts()

accommodationtype
Hotel                  266
Apartment              124
Pension                 16
Guest House              8
Hostel                   6
Bed and breakfast        4
Apart-hotel              4
Vacation home Condo      2
Name: count, dtype: int64

Selecciona solamente las filas que corresponden a hoteles. Por ejemplo: 
`df = df[df['accommodationtype'] == 'Hotel']`

In [5]:
# Selecciona las filas correspondientes a hoteles
original_df = original_df.loc[original_df['accommodationtype'] == 'Hotel']

In [6]:
# Crear copia del DataFrame original
df = original_df.copy()

## 4.2 Datos duplicados
En algunas ocasiones puede haber datos duplicados en nuestra base de datos. Para visualizar los datos duplicados podemos usar *duplicated()* de la siguiente manera:
`df[df.duplicated()]`. Si nos interesa en la variable "A" en particular entonces es `df[df['A'].duplicated()]`

In [7]:
# Identifica los datos duplicados en todas las variables 
df[df.duplicated()]

,center1distance,price,starrating,accommodationtype,guestreviewsrating,hotel_id
129,0.0,242,4.0,Hotel,4.85,22050
242,0.8,84,3.0,Hotel,2.25,22185


Una opción para eliminar datos duplicados es usar *drop_duplicates()*. En nuestro ejemplo sería: `df.drop_duplicates()`. Esto sólo elimina observaciones filas duplicadas en todas las variables. De manera predeterminada, solamente conserva la primera fila de ellas.  
En algunas ocasiones necesitaremos eliminar observaciones duplicadas en solamente algunas variables. En ese caso se puede agregar un listado con el argumento *subset*, por ejemplo: `df = df.drop_duplicates(subset=['hotel_id'])`

In [8]:
def eliminar_duplicados(df):
    """Elimina filas duplicadas y registra la cantidad eliminada"""
    n_antes = len(df)
    df = df.drop_duplicates()
    n_despues = len(df)
    print(f"Se eliminaron {n_antes - n_despues} filas duplicadas")
    return df

In [9]:
df_clean = eliminar_duplicados(df)

Se eliminaron 2 filas duplicadas


## 4.3 Datos perdidos
En el caso de datos perdidos es importante determinar si se pueden considerar aleatorios o si existe un patrón que pueda afectar los resultados. En una encuesta, por ejemplo, es posible que las personas que prefieran no mencionar su ingreso tengan ciertas características, o bien, que las personas tiendan a no responder a preguntas sobre cierto tipo de comportamiento que no es socialmente aceptable.

Es recomendable realizar pruebas estadísticas para verificar si los datos perdidos pueden ser considerados aleatorios o no. Bajo el procedimiento habitual, se construyen dos grupos, uno de observaciones con datos completos y otro grupo de observaciones con datos perdidos en una variable en particular. Posteriormente, se realizan pruebas para comparar si existen diferencias significativas en los valores promedio de las otras variables

Como una aproximación inicial, el método `info()` puede servir para identificar qué variables tienen valores perdidos.

In [10]:
# Identifica qué variables tienen valores perdidos con 'info()'
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 266 entries, 1 to 428
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   center1distance     266 non-null    float64
 1   price               266 non-null    int64  
 2   starrating          266 non-null    float64
 3   accommodationtype   266 non-null    object 
 4   guestreviewsrating  265 non-null    float64
 5   hotel_id            266 non-null    int64  
dtypes: float64(3), int64(2), object(1)
memory usage: 14.5+ KB


Para encontrar las filas con datos perdidos podemos usar `df[df.isna().any(axis=1)]`. El método *isna()* sirve para indicar qué valores son perdidos y el método *any* sirve para indicar qué filas tienen al menos un valor perdido.

In [11]:
# Identifica qué observaciones tienen datos perdidos 
df[df.isna().any(axis=1)]

,center1distance,price,starrating,accommodationtype,guestreviewsrating,hotel_id
14,0.7,106,2.5,Hotel,NaN,21916


*¿Qué hacer con datos perdidos?*  
Los métodos más comunes para tratar con datos perdidos son:
- Borrar filas o columnas con datos perdidos. Si pocas filas tienen datos perdidos (digamos, menos del 5%) y estos parecen ser completamente aleatorios, borrar las filas correspondientes es una alternativa. Si una columna tiene una gran cantidad de datos perdidos, se puede considerar la eliminación de esa columna.
- Imputación simple: Reemplazar los datos perdidos con un valor calculado a partir de la misma variable. La opción más común y conservadora es utilizar el promedio. Sin embargo, este método es muy criticado debido a que reduce la variabilidad y afecta la estimación de los intervalos de confianza (Treiman, 2009).
- Imputación multivariada: Reemplazar los datos perdidos con un valor calculado a partir de otras variables. En la imputación de datos por medio de la regresión se utilizan otras variables en la base de datos para establecer una ecuación de regresión en la que la variable dependiente es la variable con los datos perdidos. La ecuación de regresión estimada se utiliza para estimar los datos perdidos 


En cuanto a otros métodos de estimación se recomienda la lectura del capítulo “Multiple imputation of missing data” de Treiman (2009).


*Imputación simple o univariada*  
La imputación con la media para una columna 'A' es:  
`df['A'] = df['A'].fillna(df['A'].mean())`  
La imputación con la mediana es  
`df['A'] = df['A'].fillna(df['A'].median())`   
La imputación con la moda es:  
`df['A'] = df['A'].fillna(df['A'].mode().iloc[0])` 

In [12]:
# Consulta las variables que tienen datos perdidos con 'info()'
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 266 entries, 1 to 428
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   center1distance     266 non-null    float64
 1   price               266 non-null    int64  
 2   starrating          266 non-null    float64
 3   accommodationtype   266 non-null    object 
 4   guestreviewsrating  265 non-null    float64
 5   hotel_id            266 non-null    int64  
dtypes: float64(3), int64(2), object(1)
memory usage: 14.5+ KB


In [13]:
def manejar_datos_perdidos(df):
    """ Maneja valores faltantes según el tipo de datos de cada columna"""
    # Mostrar información sobre valores faltantes
    print("\nValores faltantes por columna:")
    print(df.isnull().sum())
    
    for columna in df.columns:
        # Para columnas numéricas, rellenar con la mediana
        if df[columna].dtype in ['int64', 'float64']:
            df[columna] = df[columna].fillna(df[columna].median())
            
        # Para columnas de texto, rellenar con 'DESCONOCIDO'
        elif df[columna].dtype == 'object':
            df[columna] = df[columna].fillna('DESCONOCIDO')
            
    return df

## 4.4 Valores atípicos
Un valor atípico (outlier) es un valor extremo en una o más variables. En el caso de series univariadas, es común utilizar el método del valor z o el criterio del rango intercuartil. Para el caso de datos multivariados generalmente se utilizan criterios basados en la distancia de Mahalanobis y otras técnicas multivariadas. 

*¿Qué hacer con datos atípicos?*
Si el valor extremo es un error de captura o parte de otra población lo recomendable es corregir o borrar el caso o variable. Si el valor extremo es parte de la distribución se debe mantener, pero se recomienda transformar la variable de forma tal que el valor extremo no impacte los resultados (p. ej. transformación logarítmica, transformación de Box-Cox o la recodificación de datos).

*Método del valor z*
En este enfoque un valor atípico es aquel que esté a más de tres desviaciones estándar a partir de la media.

In [14]:
def atipicos_3s(df, columna):
    """
    Elimina valores atípicos utilizando el método del valor z.
    """
    media = df[columna].mean()
    desviacion_estandar = df[columna].std()
    limite_superior = media + 3 * desviacion_estandar
    limite_inferior = media - 3 * desviacion_estandar
    df_sin_atipicos = df[(df[columna] > limite_inferior) & (df[columna] < limite_superior)]
    valores_excluidos = len(df) - len(df_sin_atipicos)
    print(f"Cantidad de valores atípicos excluidos: {valores_excluidos}")
    return df_sin_atipicos

In [15]:
atipicos_3s(df,'price')

Cantidad de valores atípicos excluidos: 6


,center1distance,price,starrating,accommodationtype,guestreviewsrating,hotel_id
1,1.7,81,4.0,Hotel,3.95,21897
2,1.4,85,4.0,Hotel,3.75,21901
3,1.7,83,3.0,Hotel,45.00,21902
4,1.2,82,4.0,Hotel,3.95,21903
6,0.9,103,4.0,Hotel,3.95,21906
...,...,...,...,...,...,...
423,1.5,95,4.0,Hotel,4.15,22402
424,1.5,73,3.0,Hotel,3.45,22403
426,0.8,185,5.0,Hotel,4.35,22406
427,1.0,100,4.0,Hotel,4.45,22407


*Método del rango intercuartil*. Otro criterio común es considerar como atípicos los valores que están a más de 1.5 veces el rango intercuartil hacia el extremo a partir del 1er. o 3er. cuartil.

In [16]:
def atipicos_iqr(df, columna):
    """
    Elimina valores atípicos utilizando el criterio del rango intercuartil.
    """
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    df_sin_atipicos = df[(df[columna] > limite_inferior) & (df[columna] < limite_superior)]
    valores_excluidos = len(df) - len(df_sin_atipicos)
    print(f"Cantidad de valores atípicos excluidos: {valores_excluidos}")
    return df_sin_atipicos

In [17]:
atipicos_iqr(df, 'price')

Cantidad de valores atípicos excluidos: 20


,center1distance,price,starrating,accommodationtype,guestreviewsrating,hotel_id
1,1.7,81,4.0,Hotel,3.95,21897
2,1.4,85,4.0,Hotel,3.75,21901
3,1.7,83,3.0,Hotel,45.00,21902
4,1.2,82,4.0,Hotel,3.95,21903
6,0.9,103,4.0,Hotel,3.95,21906
...,...,...,...,...,...,...
423,1.5,95,4.0,Hotel,4.15,22402
424,1.5,73,3.0,Hotel,3.45,22403
426,0.8,185,5.0,Hotel,4.35,22406
427,1.0,100,4.0,Hotel,4.45,22407


*Algoritmo isolation forest*

In [18]:
from sklearn.ensemble import IsolationForest

def eliminar_atipicos_isolation_forest(df, columnas, contamination=0.05, random_state=42):
    """
    Elimina valores atípicos utilizando el algoritmo Isolation Forest.
    """
    # Crear y ajusta el modelo de Isolation Forest
    iso_forest = IsolationForest(contamination=contamination, random_state=random_state)
    iso_forest.fit(df[columnas])

    # Predecir etiquetas: 1 (normal) o -1 (atípico)
    etiquetas = iso_forest.predict(df[columnas])

    # Calcular y mostrar la cantidad de valores atípicos excluidos
    df_sin_atipicos = df[etiquetas == 1]
    valores_excluidos = len(df) - len(df_sin_atipicos)
    print(f"Cantidad de valores atípicos excluidos: {valores_excluidos}")

    return df_sin_atipicos

In [19]:
datos_sin_atipicos = eliminar_atipicos_isolation_forest(df, columnas=['center1distance','price'])

Cantidad de valores atípicos excluidos: 14


## 4.5 Reporte general

In [20]:
def generar_reporte_calidad(df):
    """
    Genera un reporte básico de calidad de datos
    """
    print("\nREPORTE DE CALIDAD DE DATOS")
    print("-" * 50)
    
    # Información básica
    print(f"Dimensiones del DataFrame: {df.shape}")
    print(f"\nTipos de datos:")
    print(df.dtypes)
    
    # Valores únicos por columna
    print("\nValores únicos por columna:")
    for col in df.columns:
        print(f"{col}: {df[col].nunique()} valores únicos")
    
    # Estadísticas básicas para columnas numéricas
    print("\nEstadísticas básicas:")
    print(df.describe())

In [25]:
# Generar reporte de calidad de datos
generar_reporte_calidad(df)


REPORTE DE CALIDAD DE DATOS
--------------------------------------------------
Dimensiones del DataFrame: (250, 6)

Tipos de datos:
center1distance       float64
price                   int64
starrating            float64
accommodationtype      object
guestreviewsrating    float64
hotel_id                int64
dtype: object

Valores únicos por columna:
center1distance: 48 valores únicos
price: 125 valores únicos
starrating: 8 valores únicos
accommodationtype: 1 valores únicos
guestreviewsrating: 17 valores únicos
hotel_id: 250 valores únicos

Estadísticas básicas:
       center1distance       price  starrating  guestreviewsrating  \
count       250.000000  250.000000  250.000000          250.000000   
mean          1.502400  116.428000    3.592000           11.204400   
std           1.259627   56.059105    0.730528           15.402095   
min           0.000000   33.000000    1.000000            2.250000   
25%           0.700000   81.000000    3.000000            3.950000   
50%     

## 4.6 Exportar datos (opcional)

Un dataframe lo podemos guardar con `df.to_excel('carpeta/archivo.xlsx', index=False)`

In [21]:
# Exporta el dataframe depurado con el nombre 'output'
df.to_excel('data/output.xlsx', index=False)

## 4.6 Resumen del flujo de trabajo

In [26]:
# Crear una copia para no modificar el original
df = original_df.copy()
    
# Eliminar filas duplicadas
df = eliminar_duplicados(df)
    
# Manejar datos perdidos
df = manejar_datos_perdidos(df)

# Eliminar valores atípicos
df = eliminar_atipicos_isolation_forest(df, columnas=['center1distance','price'])

Se eliminaron 2 filas duplicadas

Valores faltantes por columna:
center1distance       0
price                 0
starrating            0
accommodationtype     0
guestreviewsrating    1
hotel_id              0
dtype: int64
Cantidad de valores atípicos excluidos: 14


## Referencias
Treiman, D. J. (2009). *Quantitative data analysis. Doing social research to test ideas*. San Francisco, CA: Jossey-Bass.